# Comparing different solver 
(**Rectangular integration** )

### Importing the Model

In [1]:
using JuMP , Ipopt
import HSL_jll
include("../Problems/JuMP/space_Shuttle_JMP.jl")

space_Shuttle_JMP (generic function with 2 methods)

## Comparing Linear Solvers : HSL_MA57, HSL_MA27 and MUMPS

#### 1. MUMPS

In [2]:
model = space_Shuttle_JMP();
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_optimizer_attribute(model,"linear_solver", "mumps")
optimize!(model);
println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Final latitude θ = 34.1790697°


#### 2. HSL_MA57

In [3]:
import HSL_jll
model = space_Shuttle_JMP();
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_attribute(model, "hsllib", HSL_jll.libhsl_path)
set_attribute(model, "linear_solver", "ma57")
optimize!(model)  # Solve for the control and state
@assert is_solved_and_feasible(model)

println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)

Final latitude θ = 34.1790697°


#### 3. HSL_MA27

In [4]:
import HSL_jll
model = space_Shuttle_JMP();
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_attribute(model, "hsllib", HSL_jll.libhsl_path)
set_attribute(model, "linear_solver", "ma27")
optimize!(model)  # Solve for the control and state
@assert is_solved_and_feasible(model)

println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)

Final latitude θ = 34.1790697°


>>>#### These 3 linear solvers give the same results for the rectangular integration problem.


## Comparing Nonlinear Solvers : Ipopt, MadNLP and Knitro

#### 1. Ipopt

> The reference solver is Ipopt with HSL_MA57 linear solver.

In [5]:
import HSL_jll
model = space_Shuttle_JMP();
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_attribute(model, "hsllib", HSL_jll.libhsl_path)
set_attribute(model, "linear_solver", "ma57")
optimize!(model)  # Solve for the control and state
println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)

Final latitude θ = 34.1790697°


#### 2. MadNLP

In [6]:
using  MadNLP
model = space_Shuttle_JMP();
set_silent(model)
set_optimizer(model,MadNLP.Optimizer)
optimize!(model)  # Solve for the control and state
println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)


Final latitude θ = 34.166426°


> the problem for not finding the same solutions with Ipopt + HSL_MA57 and MadNLP comes from the linear solvers used . MadNLP uses Umfpack by default, while we use HSL_MA57 with Ipopt.

In [7]:
using  MadNLP
using MadNLPHSL
model = space_Shuttle_JMP();
set_silent(model)
set_optimizer(model,()->MadNLP.Optimizer(linear_solver=Ma57Solver))
optimize!(model)  # Solve for the control and state
println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)

Final latitude θ = 34.1790697°
